# Wine Quality Prediction Dashboard
This demo will consume a REST API fronting the latest trained model.

In [1]:
from notebooks.src.environment import init
import pandas as pd
import panel as pn
import warnings
import numpy as np

warnings.filterwarnings("ignore")
np.random.seed(42)

experiment_id, client = init()

In [2]:
# Define our sliders

longitude_float_slider = pn.widgets.EditableFloatSlider(name="Longitude", start=-124, end=-115, step=0.1, value=-120)
latitude_float_slider = pn.widgets.EditableFloatSlider(name="Latitude", start=32, end=42, step=0.1, value=36)
housing_median_age_slider = pn.widgets.EditableFloatSlider(
    name="Housing Median Age", start=1, end=52, step=0.5, value=28
)
bedrooms_per_person_slider = pn.widgets.EditableFloatSlider(
    name="Bedrooms Per Person", start=0.1, end=1.0, step=0.01, value=0.21
)
population_slider = pn.widgets.EditableFloatSlider(name="Population", start=3, end=36000, step=360, value=1441)
households_slider = pn.widgets.EditableFloatSlider(name="Households", start=2, end=6100, step=60, value=501)
median_income_slider = pn.widgets.EditableFloatSlider(name="Median Income", start=0, end=15, step=0.15, value=4)

In [3]:
def build_blank_result_row() -> pd.DataFrame:
    return pd.DataFrame(
        [
            {
                "longitude": [],
                "latitude": [],
                "housing_median_age": [],
                "bedrooms_per_person": [],
                "population": [],
                "households": [],
                "median_income": [],
                "prediction": [],
            }
        ]
    )

In [4]:
results_df_pane = pn.widgets.DataFrame(build_blank_result_row(), width=1300)

In [5]:
def clear_btn_action(event):
    results_df_pane.value = build_blank_result_row()

In [6]:
from notebooks.src.rest import invoke_rest_endpoint
from ae5_tools import demand_env_var


def submit_btn_action(event):
    feature_data: list[dict] = [
        {
            "longitude": longitude_float_slider.value,
            "latitude": latitude_float_slider.value,
            "housing_median_age": housing_median_age_slider.value,
            "population": population_slider.value,
            "households": households_slider.value,
            "median_income": median_income_slider.value,
            "diag_coord": longitude_float_slider.value + latitude_float_slider.value,
            "bedperroom": bedrooms_per_person_slider.value,
            "prediction": [],
        }
    ]
    json_payload: dict = {"dataframe_records": feature_data}
    results: dict = invoke_rest_endpoint(
        endpoint_url=demand_env_var(name="SELF_HOSTED_MODEL_ENDPOINT"),
        input_data={"dataframe_records": feature_data},
        auth=False,
    )

    predicted_quality: float = float(round(results["predictions"][0], -5))
    feature_data[0]["prediction"] = predicted_quality

    row_df = pd.DataFrame(feature_data)

    if results_df_pane.value.loc[[0]]["prediction"][0] == []:
        results_df_pane.value = row_df
    else:
        results_df_pane.value = pd.concat([results_df_pane.value, row_df], ignore_index=True)

In [7]:
# Define our buttons
submit_btn = pn.widgets.Button(name="Submit", button_type="primary")
submit_btn.on_click(submit_btn_action)

clear_btn = pn.widgets.Button(name="Clear", button_type="primary")
clear_btn.on_click(clear_btn_action)

Watcher(inst=Button(button_type='primary', name='Clear'), cls=<class 'panel.widgets.button.Button'>, fn=<function clear_btn_action at 0x11e9db4c0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [8]:
pn.extension()

title: str = "<h1>Housing Prices Dashboard</h1>"
description = pn.pane.HTML("""Feature selection and predicted value""")

# Build layout
pn.Column(
    pn.Row(pn.Column(title, description)),
    pn.Row(
        pn.Column(
            longitude_float_slider,
            latitude_float_slider,
            housing_median_age_slider,
            bedrooms_per_person_slider,
            population_slider,
            households_slider,
            median_income_slider,
        ),
        pn.Column(pn.Row(submit_btn, clear_btn), results_df_pane),
    ),
    pn.Row(
        pn.Column(
            "### Feature Description",
            "*longitude*: ",
            "*latitude*: ",
            "*housing_median_age*: ",
            "*bedrooms_per_person*: .",
            "*households*:",
            "*median_income*: ",
        )
    ),
).servable("Housing Prices Dashboard")

Column
    [0] Row
        [0] Column
            [0] Markdown(str)
            [1] HTML(str)
    [1] Row
        [0] Column
            [0] EditableFloatSlider(end=-115, name='Longitude', start=-124, value=-120, width=300)
            [1] EditableFloatSlider(end=42, name='Latitude', start=32, value=36, width=300)
            [2] EditableFloatSlider(end=52, name='Housing Median Age', start=1, step=0.5, value=28, width=300)
            [3] EditableFloatSlider(name='Bedrooms Per Person', start=0.1, step=0.01, value=0.21, width=300)
            [4] EditableFloatSlider(end=36000, name='Population', start=3, step=360, value=1441, width=300)
            [5] EditableFloatSlider(end=6100, name='Households', start=2, step=60, value=501, width=300)
            [6] EditableFloatSlider(end=15, name='Median Income', step=0.15, value=4, width=300)
        [1] Column
            [0] Row
                [0] Button(button_type='primary', name='Submit')
                [1] Button(button_type='primary', name='Clear')
            [1] DataFrame(value=  longitude latitude h..., width=1300)
    [2] Row
        [0] Column
            [0] Markdown(str)
            [1] Markdown(str)
            [2] Markdown(str)
            [3] Markdown(str)
            [4] Markdown(str)
            [5] Markdown(str)
            [6] Markdown(str)